In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [20]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {w: i for i, w in enumerate(world_set)}

In [21]:
print(vocab)

{'m': 0, 'i': 1, ' ': 2, 'p': 3, 't': 4, 'e': 5, 'v': 6, 'l': 7, 'c': 8, 'a': 9, 'o': 10, 'C': 11, 'k': 12, 'A': 13, 'u': 14, 'n': 15, 'G': 16, 'U': 17, 'h': 18, 'P': 19, 'r': 20, 'b': 21, 'f': 22, 's': 23, 'g': 24, ',': 25, 'T': 26}


In [22]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 27


In [23]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [24]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Although t -> lthough th
1 lthough th -> though the
2 though the -> hough the 
3 hough the  -> ough the c
4 ough the c -> ugh the co
5 ugh the co -> gh the cor
6 gh the cor -> h the core
7 h the core ->  the core 
8  the core  -> the core f
9 the core f -> he core fu
10 he core fu -> e core fun
11 e core fun ->  core func
12  core func -> core funct
13 core funct -> ore functi
14 ore functi -> re functio
15 re functio -> e function
16 e function ->  function 
17  function  -> function o
18 function o -> unction of
19 unction of -> nction of 
20 nction of  -> ction of a
21 ction of a -> tion of a 
22 tion of a  -> ion of a c
23 ion of a c -> on of a ch
24 on of a ch -> n of a cha
25 n of a cha ->  of a chat
26  of a chat -> of a chatb
27 of a chatb -> f a chatbo
28 f a chatbo ->  a chatbot
29  a chatbot -> a chatbot 
30 a chatbot  ->  chatbot i
31  chatbot i -> chatbot is
32 chatbot is -> hatbot is 
33 hatbot is  -> atbot is t
34 atbot is t -> tbot is to
35 tbot is to -> bot is to 
36

In [25]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[13, 7, 4, 18, 10, 14, 24, 18, 2, 4]
[7, 4, 18, 10, 14, 24, 18, 2, 4, 18]


In [26]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(np.max(x_data) + 1)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [27]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([222, 10, 27])
레이블의 크기 : torch.Size([222, 10])


In [28]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0

In [29]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 7,  4, 18, 10, 14, 24, 18,  2,  4, 18])


In [30]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [31]:
net = Net(vocab_size, hidden_size, 2)

In [32]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [33]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([222, 10, 27])


In [34]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

lluuuubbuuuuuuubuubupbuuubuubuuubuuuubbuuuuuuuuuuuuuuuuuuuuububuumuuuuububuuuuuuuuuuubuuuuuumuuuububuuuuumuubuubuuuubbuuuuuuuuubumuuuumuuuuuumuulubuuuluuuuubuuumuuuuuuuuuuuuuuuuuumuubuumuuuuuubuuuubububbubuubuluuuubuuubuuuuuuuubuub
lrrllrlrerrrerrolmrrolrolrelrererrelolrorreroerrrrreemrrorrerrrrroreroroerrrooorrelloreerereorerorrrrrreemrrrooorelolrooerrelloreeerrrrrelererrolrrerelrrrrorerererlorrorrerrrerermmrrrrrorerorlrellorererrrorerelrerrrrorrrlrrrrrororo
lrr r rrer rerrerererrror er eeereeror eerereeeeerreeererrrerere ereror errreeeererre eerereererorrrrreeeeerrrereeror  eeeeerre eeerrrrrerererrer rereerrrrerer rereeereeeererererrerrre ererorererre erererererer errrrorrrrrrrereror 
lrree reeeeeeeee eee eeee ee eeeeeee e eeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeee ee eeeeeeeeeeee  eeeeeeeeeeeeee  e  eeeee ee eeeeeeeeeeeeeeeeee eeeeeee eeeee eeeeeeeeeeeeeeeee eeeee eeeeeeeee ee eeeee eeeeeeeeeeeree  eeeeeee eee
i  ee eeeeeeeeee eeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeee

In [35]:
predict_str

'lthough the core function of a chatbot is to mimic a human conversationalist, ChatGPT is versatileUnlike most chatbots, ChatGPT iemembers previous prompts given to it in the same conversationhatGPT suffers from multiple limitationa'

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [19]:
#1. 생성할 문장 데이터

sentence = ("Although the core function of a chatbot is to mimic a human conversationalist, ChatGPT is versatile"
            "Unlike most chatbots, ChatGPT remembers previous prompts given to it in the same conversatio"
            "ChatGPT suffers from multiple limitations")